In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
data_release='2021-04-27'

df = pd.read_csv(join("..", "data", "Phase2.1SurvivalRSummariesPublic", "ToShare", "table.auc.mice.toShare.csv"))
print(df.head())

# Rename columns
df = df.rename(columns={"calendar_month": "day"})

consistent_date = {
    '2020-Mar-Apr': "'20 Jan - '20 Apr",
    '2020-May-Jun': "'20 May - '20 Jun",
    '2020-Jul-Aug': "'20 Jul - '20 Aug",
    '2020-Sep-Oct': "'20 Sep - '20 Oct",
    '2020-Nov-2021-Jan': "'20 Nov - '21 Jan"
}

colors = ['#E79F00', '#0072B2', '#D45E00', '#CB7AA7', '#029F73', '#57B4E9']

# sites = ['META', 'BIDMC', 'FRBDX', 'NWU', 'MGB', 'UPENN', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
sites = ['META', 'APHP', 'FRBDX', 'ICSM', 'BIDMC', 'MGB', 'UCLA', 'UMICH', 'UPENN', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
site_colors = ['black', '#D45E00', '#0072B2', '#CB7AA7', '#E79F00', '#029F73', '#DBD03C', '#57B4E9', '#57B4E9', '#57B4E9', '#57B4E9', '#57B4E9']
sites = ['META', 'APHP', 'FRBDX', 'ICSM', 'UKFR', 'NWU', 'BIDMC', 'MGB', 'UCLA', 'UMICH', 'UPENN', 'UPITT', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
site_colors = ['black', '#0072B2', '#0072B2', '#0072B2', '#0072B2', '#CB7AA7', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00', '#D45E00','#D45E00','#D45E00']

df.siteid = df.siteid.apply(lambda x: x.upper())
df.day = df.day.apply(lambda x: consistent_date[x])
df.metrics = df.metrics.apply(lambda x: x.upper())

unique_days = df.day.unique().tolist()
metrics = df.metrics.unique().tolist()
print(df.metrics.unique().tolist())

df

# All Sites

In [ ]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_lab(df=None, metric='cov'):
    d = df.copy()
#     d = d[d.metrics == metric]
        
    plot = alt.Chart(
        d
    ).mark_line(
        point=point,
        size=3,
#         opacity=0.3
    ).encode(
        x=alt.X("day:N", title=None, axis=alt.Axis(labelAngle=-50, tickCount=10), scale=alt.Scale(padding=1), sort=unique_days),
        y=alt.Y("auc:Q", title=None, scale=alt.Scale(domain=[0.5,0.9])),
        # color=alt.Color("siteid:N", scale=alt.Scale(domain=sites, range=site_colors)),
        color=alt.Color("metrics:N", scale=alt.Scale(domain=metrics, range=colors), title="Metric"),
        # shape=alt.Shape("metrics:N", scale=alt.Scale(domain=metrics, range=['circle', 'diamond', 'cross', 'triangle']))
    ).properties(
        width=120,
        height=200
    )

    plot = plot.facet(
        column=alt.Column("siteid:N", header=alt.Header(title=None), sort=sites)
    ).resolve_scale(color='shared')

    plot = plot.properties(
        title={
            "text": [
                f"AUC"
            ],
            "dx": 40,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
            "subtitleColor": "gray",
        }
    )

    return plot

plot = plot_lab(df=df)

# plot = alt.vconcat(*(
#     plot_lab(df=df, lab=lab) for lab in unique_sites
# ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    header_label_orient='top',
    legend_orient='top',
    legend_title_orient='left',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot

In [ ]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_lab(df=None, metric='cov'):
    d = df.copy()
    d = d[d.siteid=='META']
#     d = d[d.metrics == metric]
        
    plot = alt.Chart(
        d
    ).mark_line(
        point=point,
        size=3,
#         opacity=0.3
    ).encode(
        x=alt.X("day:N", title=None, axis=alt.Axis(labelAngle=-50, tickCount=10), scale=alt.Scale(padding=1), sort=unique_days),
        y=alt.Y("auc:Q", title=None, scale=alt.Scale(zero=False, domain=[0.4,0.9])),
        # color=alt.Color("siteid:N", scale=alt.Scale(domain=sites, range=site_colors)),
        color=alt.Color("metrics:N", scale=alt.Scale(domain=metrics, range=colors), title="Metric", legend=None),
        # shape=alt.Shape("metrics:N", scale=alt.Scale(domain=metrics, range=['circle', 'diamond', 'cross', 'triangle']))
    ).properties(
        width=300,
        height=250
    )
    tick = plot.mark_errorbar(
        opacity=0.7 #, color='black'
    ).encode(
        x=alt.X("day:N", sort=unique_days),
        y=alt.Y("ci_l:Q"),
        y2=alt.Y2("ci_u:Q"),
        stroke=alt.value('black'),
        strokeWidth=alt.value(1)
    )
    plot= (plot+tick)

#     plot = plot.facet(
#         column=alt.Column("siteid:N", header=alt.Header(title=None), sort=sites)
#     ).resolve_scale(color='shared')

    plot = plot.properties(
        title={
            "text": [
                f"Meta-Analysis Of AUC"
            ],
            "dx": 30,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
            "subtitleColor": "gray",
        }
    )

    return plot

plot = plot_lab(df=df)

# plot = alt.vconcat(*(
#     plot_lab(df=df, lab=lab) for lab in unique_sites
# ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom-right',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
) 

plot.display()
save(plot,join("..", "result", "final-auc-mice-meta.png"), scalefactor=8.0)



## Meta Country

In [ ]:
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)

def plot_lab(df=None, metric='cov', country=None):
    d = df.copy()
    d = d[d.siteid==country]
#     d = d[d.metrics == metric]
        
    plot = alt.Chart(
        d
    ).mark_line(
        point=point,
        size=3,
#         opacity=0.3
    ).encode(
        x=alt.X("day:N", title=None, axis=alt.Axis(labelAngle=-50, tickCount=10), scale=alt.Scale(padding=1), sort=unique_days),
        y=alt.Y("auc:Q", title=None, scale=alt.Scale(zero=False, domain=[0.4,1])),
        # color=alt.Color("siteid:N", scale=alt.Scale(domain=sites, range=site_colors)),
        color=alt.Color("metrics:N", scale=alt.Scale(domain=metrics, range=colors), title="Metric", legend=None),
        # shape=alt.Shape("metrics:N", scale=alt.Scale(domain=metrics, range=['circle', 'diamond', 'cross', 'triangle']))
    ).properties(
        width=300,
        height=250
    )

#     plot = plot.facet(
#         column=alt.Column("siteid:N", header=alt.Header(title=None), sort=sites)
#     ).resolve_scale(color='shared')

    plot = plot.properties(
        title={
            "text": [
                country.replace("META-","")
            ],
            "dx": 30,
#             "subtitle": [
#                 get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#             ],
            "subtitleColor": "gray",
        }
    )
    tick = plot.mark_errorbar(
        opacity=0.7 #, color='black'
    ).encode(
        x=alt.X("day:N", sort=unique_days),
        y=alt.Y("ci_l:Q"),
        y2=alt.Y2("ci_u:Q"),
        stroke=alt.value('black'),
        strokeWidth=alt.value(1)
    )
    plot= (plot+tick)

    return plot

country_list=["META-USA", "META-FRANCE", "META-GERMANY", "META-SPAIN"]
plot = alt.hconcat(*(
     plot_lab(df=df, metric="cov",country=country) for country in country_list
 ), spacing=30)

plot = apply_theme(
    plot,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='bottom-right',
    legend_title_orient='top',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=100
)

plot
save(plot,join("..", "result", "final-auc-mice-country.png"), scalefactor=8.0)


